# XGB Classifier with Filter-Based FS

Steven Sison | March 9, 2024

## Description

This document will be used to train a model using the reduced feature set obtain by using the wrapper-based method, forward feature selection. The model will be evaluated in terms of the usual metrics (accuracy, precision, F1-score, recall) as well as the training time. The model will also be stored for future evaluation purposes.

## Training the Model

### Preliminaries

#### 1. Loading the Dataset

In [1]:
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
import seaborn as sns                   # For data visualizations
import matplotlib.pyplot as plt         # For plotting
import plotly.graph_objects as go       # For plotting
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier, DMatrix, train
from sklearn.pipeline import Pipeline
import time
from datetime import datetime
import joblib
import os
import optuna
from sklearn.metrics import mean_squared_error # or any other metric
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("../../../02_feature-engineering/final-datasets/binary_unbalanced_with_lexical-content.csv")      # Loading the dataset

dataset.head()

c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,url_type,blank_lines_count,blank_spaces_count,word_count,average_word_len,webpage_size,webpage_entropy,js_count,sus_js_count,js_eval_count,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,14,36,0,0.000000,1186,5.269303,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,415,3952,11450,4.683930,26155,4.565537,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5715,88450,420,5.833333,339327,4.656704,13,10,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1461,6292,3049,4.515907,89249,5.149580,11,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,43,273,0,0.000000,1530,4.754726,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
dataset.head()

,url_type,blank_lines_count,blank_spaces_count,word_count,average_word_len,webpage_size,webpage_entropy,js_count,sus_js_count,js_eval_count,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,14,36,0,0.000000,1186,5.269303,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,415,3952,11450,4.683930,26155,4.565537,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5715,88450,420,5.833333,339327,4.656704,13,10,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1461,6292,3049,4.515907,89249,5.149580,11,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,43,273,0,0.000000,1530,4.754726,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
x_train, x_test, y_train, y_test = train_test_split(dataset.drop(columns=['url_type']), dataset['url_type'], test_size = 0.2, random_state=42)

#### 2. Preprocessing (Balancing)

In [4]:
dataset['url_type'].value_counts()

url_type
0    120080
1     62334
Name: count, dtype: int64

#### 3. Removing Unnecessary Features

In [5]:
important_features_lexical = ['url_host_length',
                              'url_is_https',  
                              'url_num_periods',
                              'url_path_length',
                              'js_count',
                              'has_log_in_html',
                              'meta_tag_count',
                              'js_search_count',
                              'url_num_forward_slash',
                              'sus_js_count',
                              'js_link_count',
                              'webpage_entropy',
                              'url_num_ampersand',
                              'url_num_subdomain',
                              'webpage_size',
                              'has_free_in_html',
                              'has_php_in_string',
                              'url_num_of_hyphens',
                              'url_query_length',
                              'get_tld',
                              'url_scheme']

X_test_lexical_content = x_test[important_features_lexical]
X_train_lexical_content = x_train[important_features_lexical]

print("Lexical+Content-Based XGB Model has a total of "+str(len(important_features_lexical))+" features.")

Lexical+Content-Based XGB Model has a total of 21 features.


#### Hyper-parameter Optimization

In [6]:
# Define the objective function for Optuna
def objective_lexical_content(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_lexical_content, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_lexical_content = optuna.create_study(direction='minimize')
study_lexical_content.optimize(objective_lexical_content, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_lexical_content = study_lexical_content.best_params
best_error_lexical_content = study_lexical_content.best_value
print("Best Hyperparameters (33 Features): ", best_params_lexical_content)
print("Best Error (33 Features): ", best_error_lexical_content)

[I 2024-04-02 13:15:17,113] A new study created in memory with name: no-name-fd0287d0-2ee8-4d22-b338-f79e6dab5a76


[0]	validation-error:0.65995
[1]	validation-error:0.65995
[2]	validation-error:0.65683
[3]	validation-error:0.24004
[4]	validation-error:0.18789


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [13:15:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[5]	validation-error:0.17597
[6]	validation-error:0.15404
[7]	validation-error:0.13359
[8]	validation-error:0.10052
[9]	validation-error:0.10083
[10]	validation-error:0.09610
[11]	validation-error:0.09545
[12]	validation-error:0.09347
[13]	validation-error:0.09062
[14]	validation-error:0.09079
[15]	validation-error:0.09014
[16]	validation-error:0.08870
[17]	validation-error:0.08956
[18]	validation-error:0.08764
[19]	validation-error:0.08754
[20]	validation-error:0.08761
[21]	validation-error:0.08627
[22]	validation-error:0.08661
[23]	validation-error:0.08648
[24]	validation-error:0.08545
[25]	validation-error:0.08422
[26]	validation-error:0.08404
[27]	validation-error:0.08356
[28]	validation-error:0.08216
[29]	validation-error:0.08161
[30]	validation-error:0.08158
[31]	validation-error:0.08117
[32]	validation-error:0.08000
[33]	validation-error:0.07993
[34]	validation-error:0.07747
[35]	validation-error:0.07699
[36]	validation-error:0.07572
[37]	validation-error:0.07514
[38]	validation

[I 2024-04-02 13:15:23,559] Trial 0 finished with value: 0.2157818932863543 and parameters: {'eta': 0.2069185763712498, 'max_depth': 3, 'subsample': 0.732808535536775, 'colsample_bytree': 0.5338231138458792, 'gamma': 3.859224125978178, 'min_child_weight': 4.623991397095878, 'lambda': 7.384862731691721, 'alpha': 7.32042329305727}. Best is trial 0 with value: 0.2157818932863543.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [13:15:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


...

[2]	validation-error:0.14325


[I 2024-04-02 13:18:01,992] Trial 98 pruned. Trial was pruned at iteration 2.


[0]	validation-error:0.65995
[1]	validation-error:0.65995


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [13:18:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-04-02 13:18:02,401] Trial 99 pruned. Trial was pruned at iteration 1.


Best Hyperparameters (33 Features):  {'eta': 0.24920171651018697, 'max_depth': 8, 'subsample': 0.7969243749302927, 'colsample_bytree': 0.7025268248381582, 'gamma': 0.23722771541916587, 'min_child_weight': 1.9542417160152912, 'lambda': 1.7466848789938463, 'alpha': 5.8218178623611525}
Best Error (33 Features):  0.18885593419636032


#### Model Training

In [20]:
from sklearn.model_selection import KFold
from xgboost import DMatrix, train

# Initialize CV
cv = KFold(n_splits=10, shuffle=True, random_state=1)

best_params_lexical_content['objective'] = 'binary:hinge'
best_params_lexical_content['eval_metric'] = 'error'

# Convert the data into DMatrix format
lexical_content_train = DMatrix(X_train_lexical_content, label=y_train)
lexical_content_valid = DMatrix(X_test_lexical_content, label=y_test)

# Train the Model
xgb_classifier = train(best_params_lexical_content, lexical_content_train, num_boost_round=1000)
y_pred_lexical_content = xgb_classifier.predict(lexical_content_valid)

print("Model training done.")

Model training done.


In [21]:
# Classification Report
print(classification_report(y_test, y_pred_lexical_content))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97     23942
           1       0.95      0.93      0.94     12541

    accuracy                           0.96     36483
   macro avg       0.96      0.95      0.96     36483
weighted avg       0.96      0.96      0.96     36483



#### Practical Evaluation

In [22]:
# Dumping the model
joblib.dump(xgb_classifier, 'xgb_filter_lexical-content.sav')

['xgb_filter_lexical-content.sav']

In [28]:
import content_generator
import feature_generation_content_function_htmlin
import time

def xgb_predict_maliciousness(url):

    numerical_values = content_generator.feature_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"

url = "www.mizuhubasnk.top"
print("Current URL: "+url)

start = time.perf_counter()
prediction = xgb_predict_maliciousness(url)
end = time.perf_counter()
print("------- Lexical+Content-Based Model -------------")
print(prediction)
print(end-start)

Current URL: www.mizuhubasnk.top


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:js_count: object, has_log_in_html: object, meta_tag_count: object, js_search_count: object, sus_js_count: object, js_link_count: object, webpage_entropy: object, webpage_size: object, has_free_in_html: object

#### Evaluation

In [11]:
'''# Confusion Matrix for 12 Features
cm_up = confusion_matrix(y_test, y_pred, labels=xgb_classifier.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier.classes_)
disp.plot()
plt.show()'''

'# Confusion Matrix for 12 Features\ncm_up = confusion_matrix(y_test, y_pred, labels=xgb_classifier.classes_)\ndisp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier.classes_)\ndisp.plot()\nplt.show()'

In [12]:
'''# Cross Validation Score
scores = cross_val_score(XGBClassifier(**params_gbm),
                        X_train, y_train, scoring='accuracy', cv=cv).mean()

print(scores)'''

"# Cross Validation Score\nscores = cross_val_score(XGBClassifier(**params_gbm),\n                        X_train, y_train, scoring='accuracy', cv=cv).mean()\n\nprint(scores)"